In [162]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from collections import defaultdict

import locale
locale.setlocale(locale.LC_ALL, 'nl_be')

'nl_be'

In [3]:
# Get index page from website
page = requests.get('https://tijdlijn-coronavirus.nl')
soup = BeautifulSoup(page.text, 'html.parser')


In [156]:
months_soup = soup.find(id='block-04fa3293-b56b-431d-b240-4663bb411f0a').find_all('a')
months = [el.attrs['href'] for el in months_soup]
months

AttributeError: 'NoneType' object has no attribute 'find_all'

In [157]:
day_data = defaultdict(dict)
for month in months[1:]:
    m_str = month.split('-')[1].split('/')[-1]
    y_str = month.split('-')[2]
    
    page = requests.get(month)
    soup = BeautifulSoup(page.text, 'html.parser').find(class_='site-column-wrapper')
    
    title = soup.find(class_='page-title').get_text()
    content = soup.find(class_='entry-content')

    for el in content.find_all(['em']):
        el.unwrap()
    for el in content.find_all(['figure', 'iframe', 'h3', 'br']):
        el.decompose()
    for el in content.find_all(class_=['wp-block-group']):
        el.decompose()
    for el in content.find_all(id=['post-modified-info']):
        el.decompose()

    # Find first introductory paragraph and remove. 
    content.find_all('p')[0].decompose()

    # Loop over possible daily entries
    for item in content.find_all(['li']): #, 'p']):

        d_str = item.strong.get_text().split(':')[0]
        print(d_str, y_str)

        for strong in item.find_all('strong'):
            strong.decompose()
        
        out = str(item).replace('\xa0', ' ').replace('. .', '')
        out = out.replace('<li>', '').replace('</li>', '')

        day_data[f"{d_str} {y_str}"] = out

31 december 2020
30 december 2020
29 december 2020
28 december 2020
24 december 2020
23 december 2020
22 december 2020
21 december 2020
20 december 2020
19 december 2020
18 december 2020
17 december 2020
16 december 2020
15 december 2020
14 december 2020
13 december 2020
12 december 2020
11 december 2020
10 december 2020
9 december 2020
8 december 2020
7 december 2020
6 december 2020
5 december 2020
4 december 2020
3 december 2020
2 december 2020
1 december 2020
30 november 2020
29 november 2020
28 november 2020
27 november 2020
26 november 2020
25 november 2020
24 november 2020
23 november 2020
22 november 2020
21 november 2020
20 november 2020
19 november 2020
18 november 2020
17 november 2020
16 november 2020
15 november 2020
14 november 2020
13 november 2020
12 november 2020
11 november 2020
10 november 2020
9 november 2020
8 november 2020
7 november 2020
6 november 2020
5 november 2020
4 november 2020
3 november 2020
2 november 2020
1 november 2020
31 oktober 2020
30 oktober 2020


In [174]:
df = pd.DataFrame.from_dict(day_data, orient='index').reset_index()
df.columns = ['date', 'text']
df = df[df.date.str.match('\d{1,2} \w+ \d{4}$')]
df.date = pd.to_datetime(df.date, format="%d %B %Y")
df = df.set_index('date')
df = df.sort_index(ascending=True)
df

,text
date,
2020-01-06,"De NOS maakt melding van een <a href=""https://..."
2020-01-09,"Het is duidelijk dat de <a aria-label=""undefin..."
2020-01-14,WHO meldt dat het virus niet besmettelijk is. ...
2020-01-17,"Het Erasmus MC is druk met het <a aria-label=""..."
2020-01-18,Vijf werknemers van een bedrijf in Sittard ker...
...,...
2020-12-24,"In Tiel is een feest <a href=""https://www.omr..."
2020-12-28,"Het RIVM <a href=""https://www.rtvutrecht.nl/n..."
2020-12-29,Iedereen die naar Nederland vliegt vanuit een...


In [175]:
df.to_csv('./data-covid-tijdlijn.csv')